## First Optimization
### Quadratic Function Example
https://optuna.readthedocs.io/en/stable/tutorial/first.html

In [1]:
import optuna

In [2]:
def objective(trial):
    # (x-2)^2を最小化するxを見つけたい
    # trialってのが鍵
    x = trial.suggest_uniform('x', -10, 10)
    return (x - 2) ** 2

In [3]:
study = optuna.create_study() #おまじない
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2018-12-25 22:12:46,355] Finished a trial resulted in value: 132.65130344073157. Current best value is 132.65130344073157 with parameters: {'x': -9.517434759560462}.
[I 2018-12-25 22:12:46,366] Finished a trial resulted in value: 8.866938909423599. Current best value is 8.866938909423599 with parameters: {'x': 4.9777405712089156}.
[I 2018-12-25 22:12:46,373] Finished a trial resulted in value: 0.06522124032361255. Current best value is 0.06522124032361255 with parameters: {'x': 2.255384495072846}.
[I 2018-12-25 22:12:46,382] Finished a trial resulted in value: 48.6901007487619. Current best value is 0.06522124032361255 with parameters: {'x': 2.255384495072846}.
[I 2018-12-25 22:12:46,389] Finished a trial resulted in value: 0.3390955911628619. Current best value is 0.06522124032361255 with parameters: {'x': 2.255384495072846}.
[I 2018-12-25 22:12:46,402] Finished a trial resulted in value: 1.5342612804614586. Current best value is 0.06522124032361255 with parameters: {'x': 2.2553844

In [4]:
# 結果
study.best_params

{'x': 2.0022914056096104}

In [5]:
study.best_value

5.250539667753946e-06

In [6]:
study.best_trial

FrozenTrial(trial_id=65, state=<TrialState.COMPLETE: 1>, value=5.250539667753946e-06, datetime_start=datetime.datetime(2018, 12, 25, 22, 12, 47, 310262), datetime_complete=datetime.datetime(2018, 12, 25, 22, 12, 47, 314403), params={'x': 2.0022914056096104}, user_attrs={}, system_attrs={}, intermediate_values={}, params_in_internal_repr={'x': 2.0022914056096104})

In [7]:
print(study.trials[99])
len(study.trials)

FrozenTrial(trial_id=99, state=<TrialState.COMPLETE: 1>, value=16.47625659184242, datetime_start=datetime.datetime(2018, 12, 25, 22, 12, 47, 979174), datetime_complete=datetime.datetime(2018, 12, 25, 22, 12, 47, 985954), params={'x': 6.059095538644344}, user_attrs={}, system_attrs={}, intermediate_values={}, params_in_internal_repr={'x': 6.059095538644344})


100

さらに続けて学習を行うことも可能

In [8]:
study.optimize(objective, n_trials=100, n_jobs=1)
len(study.trials)

[I 2018-12-25 22:12:50,837] Finished a trial resulted in value: 33.06498742230397. Current best value is 5.250539667753946e-06 with parameters: {'x': 2.0022914056096104}.
[I 2018-12-25 22:12:50,858] Finished a trial resulted in value: 6.508646049876874. Current best value is 5.250539667753946e-06 with parameters: {'x': 2.0022914056096104}.
[I 2018-12-25 22:12:50,884] Finished a trial resulted in value: 15.955732593974155. Current best value is 5.250539667753946e-06 with parameters: {'x': 2.0022914056096104}.
[I 2018-12-25 22:12:50,906] Finished a trial resulted in value: 9.859326110701508. Current best value is 5.250539667753946e-06 with parameters: {'x': 2.0022914056096104}.
[I 2018-12-25 22:12:50,929] Finished a trial resulted in value: 91.9751168488851. Current best value is 5.250539667753946e-06 with parameters: {'x': 2.0022914056096104}.
[I 2018-12-25 22:12:50,953] Finished a trial resulted in value: 42.285560158748325. Current best value is 5.250539667753946e-06 with parameters: 

200

In [9]:
study.best_params

{'x': 2.0022914056096104}

## Advanced Configurations
https://optuna.readthedocs.io/en/stable/tutorial/configurations.html#defining-parameter-spaces
### Defining Parameter Spaces
5種のハイパーパラメーターをサポートしている

In [10]:
def objective(trial):
    # Categorical parameter
    # 最適化手法やそもそものアルゴリズム選択にも使えそう
    optimizer = trial.suggest_categorical('optimizer', ['MomentumSGD', 'Adam'])

    # Int parameter
    # 離散的なパラメーター
    num_layers = trial.suggest_int('num_layers', 1, 3)

    # Uniform parameter
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 1.0)

    # Loguniform parameter
    # 指数的に変わるのでこっちの方がハイパーパラメータを探すときは使いそう
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)

    # Discrete-uniform parameter
    # どういうときに使うのかよくわからん
    drop_path_rate = trial.suggest_discrete_uniform('drop_path_rate', 0.0, 1.0, 0.1)

    ...

### Branches and Loops
条件分岐や繰り返しをハイパーパラメータにする例

In [11]:
def objective(trial):
    # そもそも何のクラシファイアーを使うのかも最適化する
    classifier_name = trial.suggest_categorical('classifier', ['SVC', 'RandomForest'])
    if classifier_name == 'SVC':
        # 注目すべきは、分岐したあとにもtrialでハイパーパラメータをサーチできることだ
        svc_c = trial.suggest_loguniform('svc_c', 1e-10, 1e10)
        classifier_obj = sklearn.svm.SVC(C=svc_c)
    else:
        # ここもそう。動的にサーチするパラメーターを決めることができるって点でdefine by run
        rf_max_depth = int(trial.suggest_loguniform('rf_max_depth', 2, 32))
        classifier_obj = sklearn.ensemble.RandomForestClassifier(max_depth=rf_max_depth)

    ...

In [12]:
def create_model(trial):
    # ここでレイヤーの数を決めておいて
    n_layers = trial.suggest_int('n_layers', 1, 3)

    layers = []
    for i in range(n_layers):
        # ここでノードの数も決められる！
        n_units = int(trial.suggest_loguniform('n_units_l{}'.format(i), 4, 128))
        layers.append(L.Linear(None, n_units))
        layers.append(F.relu)
    layers.append(L.Linear(None, 10))
    #めっっちゃ動的に最適な構造を探せる!!!
    return chainer.Sequential(*layers)

### パラメータの数について
最適化の難易度はハイパーパラメータの数に対して、指数的に難しくなる。
そのためtrialの数も指数的に増やしていかなければならない。
重要じゃないパラメーラー入れないことをおすすめする。

### study.optimize()の引数について
`n_trials`と`timeout`を指定することができる。
ちなみに何も入れないとctrl-Cで終了するまで最適化を続ける。

## RDBを用いた保存や分散処理等
https://optuna.readthedocs.io/en/stable/tutorial/rdb.html
RDBの知識がないので一回スキップ(勉強しなきゃね)

## Pruning Unpromising Trials(見込みのない試行の枝刈り
https://optuna.readthedocs.io/en/stable/tutorial/pruning.html

### Activating Pruners
